In [1]:
import pandas as pd
import geopandas as gp
import numpy as np
import os
import math
import zipfile

In [2]:
# DONT RUN
for filename in os.listdir('../raw/slope-aspect/'):
    if filename.endswith('.zip'):
        zip_f = zipfile.ZipFile("../raw/slope-aspect/"+filename, "r")
        f = filename.strip("NASADEM_SC_").strip(".zip")
#         zip_f.extract("../raw/slope-aspect/hgt/"+f+".hgt")
        zip_f.extract(f+".slope")
        zip_f.close()
        print(filename)

NASADEM_SC_n12e120.zip
NASADEM_SC_n20e116.zip
NASADEM_SC_n06e115.zip
NASADEM_SC_n13e124.zip
NASADEM_SC_n12e121.zip
NASADEM_SC_n12e123.zip
NASADEM_SC_n06e117.zip
NASADEM_SC_n06e116.zip
NASADEM_SC_n12e122.zip
NASADEM_SC_n13e123.zip
NASADEM_SC_n07e117.zip
NASADEM_SC_n07e116.zip
NASADEM_SC_n21e114.zip
NASADEM_SC_n13e122.zip
NASADEM_SC_n12e119.zip
NASADEM_SC_n12e125.zip
NASADEM_SC_n13e120.zip
NASADEM_SC_n13e121.zip
NASADEM_SC_n12e124.zip
NASADEM_SC_n09e125.zip
NASADEM_SC_n09e119.zip
NASADEM_SC_n18e120.zip
NASADEM_SC_n05e117.zip
NASADEM_SC_n10e126.zip
NASADEM_SC_n11e123.zip
NASADEM_SC_n11e122.zip
NASADEM_SC_n05e116.zip
NASADEM_SC_n18e121.zip
NASADEM_SC_n09e118.zip
NASADEM_SC_n09e124.zip
NASADEM_SC_n09e126.zip
NASADEM_SC_n05e114.zip
NASADEM_SC_n08e123.zip
NASADEM_SC_n10e125.zip
NASADEM_SC_n10e119.zip
NASADEM_SC_n11e120.zip
NASADEM_SC_n11e121.zip
NASADEM_SC_n10e118.zip
NASADEM_SC_n10e124.zip
NASADEM_SC_n18e122.zip
NASADEM_SC_n05e115.zip
NASADEM_SC_n08e122.zip
NASADEM_SC_n09e123.zip
NASADEM_SC_

In [2]:
file_path = '../raw/slope-aspect/slope-raw/'

In [4]:
for filename in os.listdir(file_path):
    if filename.endswith('.slope'):
        filename_noext = filename.strip('.slope')
        os.rename(file_path+filename, file_path+filename_noext+'.hgt')

In [7]:
final = pd.DataFrame()
for filename in os.listdir(file_path):
    if filename.endswith('.hgt'):
        lons = []
        lats = []
        lon_tile = int(filename[4:7])
        lat_tile = int(filename[1:3])

        siz = os.path.getsize(file_path + filename)
        dim = int(math.sqrt(siz/2))

        assert dim*dim*2 == siz, 'Invalid file size'

        data = np.fromfile(file_path + filename, np.dtype('>i2'), dim*dim).reshape((dim, dim))

        for i in range(0, 3601):
            lon = lon_tile + 1/7200 + i*(1/3601)
            lat = lat_tile + 1/7200 + i*(1/3601)
            lons.append(lon)
            lats.append(lat)

        tile_df = pd.DataFrame(data)
        tile_df.index = lats
        tile_df = tile_df.T
        tile_df.index = lons
        tile_df = tile_df.T
        
        tile_df = tile_df.iloc[::50, ::50]        
        tile_final = tile_df.stack().reset_index()
        tile_final = tile_final.rename(columns = {'level_0': 'lat', 'level_1': 'lon', 0: 'slope'})
        
        final = final.append(tile_final)
        print(filename)

n15e122.hgt
n04e126.hgt
n11e115.hgt
n08e116.hgt
n05e121.hgt
n15e121.hgt
n14e124.hgt
n04e118.hgt
n04e119.hgt
n04e125.hgt
n15e120.hgt
n08e117.hgt
n05e120.hgt
n11e114.hgt
n10e115.hgt
n05e118.hgt
n05e124.hgt
n14e121.hgt
n09e117.hgt
n14e120.hgt
n05e125.hgt
n15e119.hgt
n05e119.hgt
n10e114.hgt
n14e122.hgt
n14e123.hgt
n05e126.hgt
n07e124.hgt
n07e118.hgt
n06e121.hgt
n16e121.hgt
n20e122.hgt
n16e120.hgt
n06e120.hgt
n07e125.hgt
n16e122.hgt
n06e122.hgt
n20e121.hgt
n06e123.hgt
n07e126.hgt
n21e120.hgt
n07e122.hgt
n17e122.hgt
n06e126.hgt
n07e123.hgt
n21e121.hgt
n17e121.hgt
n07e121.hgt
n06e118.hgt
n06e124.hgt
n06e125.hgt
n16e119.hgt
n17e120.hgt
n20e116.hgt
n12e120.hgt
n12e121.hgt
n13e124.hgt
n06e115.hgt
n06e117.hgt
n12e123.hgt
n12e122.hgt
n06e116.hgt
n07e117.hgt
n13e123.hgt
n13e122.hgt
n07e116.hgt
n21e114.hgt
n13e120.hgt
n12e119.hgt
n12e125.hgt
n12e124.hgt
n13e121.hgt
n11e123.hgt
n10e126.hgt
n05e117.hgt
n18e120.hgt
n09e125.hgt
n09e119.hgt
n09e118.hgt
n09e124.hgt
n18e121.hgt
n05e116.hgt
n11e122.hgt
n11e

In [8]:
final = final[final['slope'] >= 0]

In [9]:
final

,lat,lon,slope
0,15.000139,122.000139,0
1,15.000139,122.014024,0
2,15.000139,122.027909,0
3,15.000139,122.041794,0
4,15.000139,122.055679,0
...,...,...,...
5324,10.999861,122.944321,1686
5325,10.999861,122.958206,2902
5326,10.999861,122.972091,2914
5327,10.999861,122.985976,2071


In [10]:
final.to_csv('../raw/slope-aspect/slope-vector/slope_50.csv', index = False)